In [46]:
import requests
import pandas as pd
import config
import json
import time
import gmaps
import pytz

In [56]:
#install and import 'pycountry' - a package that relates country codes to country names
#helps with geocoding using Google's API
!pip install pycountry
import pycountry

  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746867 sha256=31a92d5bbb1e0a6287b36cdf86033e442fd88be67f63deffc50a4057651ddc8f
  Stored in directory: c:\users\patri\appdata\local\pip\cache\wheels\57\e8\3f\120ccc1ff7541c108bc5d656e2a14c39da0d824653b62284c6
Successfully built pycountry


# Scrape Coordinates

* Notebook uses Google Geocode API to get info about ~2000 city coordinates (latitude, longitude) where startups are located

* Free monthly limit of about 40,000 API calls from automatic free $200 credit/month

* CSV and JSON output files of city and coordinate data - CSV format for use in other notebooks with pandas, and JSON format for preliminary setup and test runs of geographical mapping in Leaflet

In [2]:
#test that everything is working correctly on an example request
base_url = "https://maps.googleapis.com/maps/api/geocode/json?address="

city = "San Francisco"
state_code = "CA"
country_code = "USA"

response = requests.get("{}{}+{}+{}&key={}".format(base_url, city, state_code, country_code, config.API_KEY))

print(json.dumps(response.json(), indent=4))
print(response.json()["results"][0]["geometry"]["location"])

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "San Francisco",
                    "short_name": "SF",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "San Francisco County",
                    "short_name": "San Francisco County",
                    "types": [
                        "administrative_area_level_2",
                        "political"
                    ]
                },
                {
                    "long_name": "California",
                    "short_name": "CA",
                    "types": [
                        "administrative_area_level_1",
                        "political"
                    ]
                },
                {
                    "long_name": "United States",
                    "short_name": "US",
           

In [3]:
#read in CSV containing unique city, state, and country data
valid_locations_df = pd.read_csv("./geographical_data/all_places.csv")
valid_locations_df

,country_code,state_code,city
0,USA,NY,New York
1,USA,IL,Champaign
2,USA,CA,Los Angeles
3,USA,NJ,Iselin
4,USA,MD,Woodbine
...,...,...,...
2168,CAN,ON,Paris
2169,USA,FL,Atlantic Beach
2170,USA,OR,Salem
2171,USA,WA,Auburn


In [32]:
#initialize copy of data frame to store data
coords_df = valid_locations_df.copy()
coords_df["Latitude"] = None
coords_df["Longitude"] = None
coords_df

,country_code,state_code,city,Latitude,Longitude
0,USA,NY,New York,None,None
1,USA,IL,Champaign,None,None
2,USA,CA,Los Angeles,None,None
3,USA,NJ,Iselin,None,None
4,USA,MD,Woodbine,None,None
...,...,...,...,...,...
2168,CAN,ON,Paris,None,None
2169,USA,FL,Atlantic Beach,None,None
2170,USA,OR,Salem,None,None
2171,USA,WA,Auburn,None,None


In [33]:
#write for loop to send requests each second, and append data to a copy of the read dataframe
for i in range(len(coords_df)):
    city = coords_df.iloc[i]["city"]
    state = coords_df.iloc[i]["state_code"]
    country = coords_df.iloc[i]["country_code"]
    try:
        response = requests.get("{}{}+{}+{}&key={}".format(base_url, city, state, country, config.API_KEY))
        results = response.json()
        print(results["results"][0]["geometry"]["location"])
        lat = results["results"][0]["geometry"]["location"]["lat"]
        lng = results["results"][0]["geometry"]["location"]["lng"]
        time.sleep(1)
        coords_df["Latitude"][i] = lat
        coords_df["Longitude"][i] = lng
        print("Working response for {}, {}, {} with results: {}, {}".format(city, state, country, lat, lng))
    except:
        print("Issue with {}, {}, {}".format(city, state, country))


{'lat': 40.7127753, 'lng': -74.0059728}
Working response for New York, NY, USA with results: 40.7127753, -74.0059728
{'lat': 40.1164204, 'lng': -88.2433829}
Working response for Champaign, IL, USA with results: 40.1164204, -88.2433829
{'lat': 34.0522342, 'lng': -118.2436849}
Working response for Los Angeles, CA, USA with results: 34.0522342, -118.2436849
{'lat': 40.5753817, 'lng': -74.3223703}
Working response for Iselin, NJ, USA with results: 40.5753817, -74.3223703
{'lat': 39.3738173, 'lng': -77.0569038}
Working response for Woodbine, MD, USA with results: 39.3738173, -77.0569038
{'lat': 45.415734, 'lng': -122.7163273}
Working response for Lake Oswego, OR, USA with results: 45.415734, -122.7163273
{'lat': 37.7749295, 'lng': -122.4194155}
Working response for San Francisco, CA, USA with results: 37.7749295, -122.4194155
{'lat': 47.6101497, 'lng': -122.2015159}
Working response for Bellevue, WA, USA with results: 47.6101497, -122.2015159
{'lat': 43.0730517, 'lng': -89.4012302}
Working 

Working response for Santa Clara, CA, USA with results: 37.3541079, -121.9552356
{'lat': 38.84622359999999, 'lng': -77.30637329999999}
Working response for Fairfax, VA, USA with results: 38.84622359999999, -77.30637329999999
{'lat': 42.3736158, 'lng': -71.10973349999999}
Working response for Cambridge, MA, USA with results: 42.3736158, -71.10973349999999
{'lat': 30.267153, 'lng': -97.7430608}
Working response for Austin, TX, USA with results: 30.267153, -97.7430608
{'lat': 39.9611755, 'lng': -82.99879419999999}
Working response for Columbus, OH, USA with results: 39.9611755, -82.99879419999999
{'lat': 45.6956723, 'lng': -121.8845241}
Working response for Stevenson, WA, USA with results: 45.6956723, -121.8845241
{'lat': 39.2903848, 'lng': -76.6121893}
Working response for Baltimore, MD, USA with results: 39.2903848, -76.6121893
{'lat': 42.3709299, 'lng': -71.1828321}
Working response for Watertown, MA, USA with results: 42.3709299, -71.1828321
{'lat': 33.7514966, 'lng': -84.7477136}
Wor

{'lat': 48.7519112, 'lng': -122.4786854}
Working response for Bellingham, WA, USA with results: 48.7519112, -122.4786854
{'lat': 38.2526647, 'lng': -85.7584557}
Working response for Louisville, KY, USA with results: 38.2526647, -85.7584557
{'lat': 38.8338816, 'lng': -104.8213634}
Working response for Colorado Springs, CO, USA with results: 38.8338816, -104.8213634
{'lat': 27.950575, 'lng': -82.4571776}
Working response for Tampa, FL, USA with results: 27.950575, -82.4571776
{'lat': 33.6845673, 'lng': -117.8265049}
Working response for Irvine, CA, USA with results: 33.6845673, -117.8265049
{'lat': 40.0992294, 'lng': -83.1140771}
Working response for Dublin, OH, USA with results: 40.0992294, -83.1140771
{'lat': 39.9205411, 'lng': -105.0866504}
Working response for Broomfield, CO, USA with results: 39.9205411, -105.0866504
{'lat': 41.9750287, 'lng': -88.0072909}
Working response for Itasca, IL, USA with results: 41.9750287, -88.0072909
{'lat': 39.327962, 'lng': -120.1832533}
Working respo

Working response for Laguna Hills, CA, USA with results: 33.5916433, -117.6986604
{'lat': 21.3069444, 'lng': -157.8583333}
Working response for Honolulu, HI, USA with results: 21.3069444, -157.8583333
{'lat': 28.0780718, 'lng': -82.7637127}
Working response for Palm Harbor, FL, USA with results: 28.0780718, -82.7637127
{'lat': 34.1477849, 'lng': -118.1445155}
Working response for Pasadena, CA, USA with results: 34.1477849, -118.1445155
{'lat': 38.3242355, 'lng': -85.4724593}
Working response for Crestwood, KY, USA with results: 38.3242355, -85.4724593
{'lat': 35.2270869, 'lng': -80.8431267}
Working response for Charlotte, NC, USA with results: 35.2270869, -80.8431267
{'lat': 42.0044757, 'lng': -87.73005940000002}
Working response for Lincolnwood, IL, USA with results: 42.0044757, -87.73005940000002
{'lat': 38.8942786, 'lng': -77.43109919999999}
Working response for Chantilly, VA, USA with results: 38.8942786, -77.43109919999999
{'lat': 41.0234304, 'lng': -74.06652969999999}
Working res

{'lat': 38.440429, 'lng': -122.7140548}
Working response for Santa Rosa, CA, USA with results: 38.440429, -122.7140548
{'lat': 37.8590937, 'lng': -122.4852507}
Working response for Sausalito, CA, USA with results: 37.8590937, -122.4852507
{'lat': 48.4284207, 'lng': -123.3656444}
Working response for Victoria, BC, CAN with results: 48.4284207, -123.3656444
{'lat': 41.8397865, 'lng': -87.95355339999999}
Working response for Oak Brook, IL, USA with results: 41.8397865, -87.95355339999999
{'lat': 40.6781784, 'lng': -73.9441579}
Working response for Brooklyn, NY, USA with results: 40.6781784, -73.9441579
{'lat': 40.8270448, 'lng': -73.97569399999999}
Working response for Edgewater, NJ, USA with results: 40.8270448, -73.97569399999999
{'lat': 40.5939636, 'lng': -74.6049061}
Working response for Bridgewater, NJ, USA with results: 40.5939636, -74.6049061
{'lat': 39.6477653, 'lng': -104.9877597}
Working response for Englewood, CO, USA with results: 39.6477653, -104.9877597
{'lat': 49.2488091, '

...

Working response for Quakertown, PA, USA with results: 40.4417682, -75.3415667
{'lat': 34.1397292, 'lng': -118.0353449}
Working response for Arcadia, CA, USA with results: 34.1397292, -118.0353449
{'lat': 41.18454149999999, 'lng': -73.1331651}
Working response for Stratford, CT, USA with results: 41.18454149999999, -73.1331651
{'lat': 40.3561805, 'lng': -80.1100556}
Working response for Bridgeville, PA, USA with results: 40.3561805, -80.1100556
{'lat': 41.4768217, 'lng': -81.8497567}
Working response for Rocky River, OH, USA with results: 41.4768217, -81.8497567
{'lat': 42.4795019, 'lng': -79.3339324}
Working response for Dunkirk, NY, USA with results: 42.4795019, -79.3339324
{'lat': 41.12086679999999, 'lng': -88.835352}
Working response for Streator, IL, USA with results: 41.12086679999999, -88.835352
{'lat': 41.2306979, 'lng': -73.064036}
Working response for Milford, CT, USA with results: 41.2306979, -73.064036
{'lat': 46.73238749999999, 'lng': -117.000165}
Working response for Mosc

{'lat': 40.7998227, 'lng': -73.6509622}
Working response for Roslyn, NY, USA with results: 40.7998227, -73.6509622
{'lat': 41.5623209, 'lng': -72.6506488}
Working response for Middletown, CT, USA with results: 41.5623209, -72.6506488
{'lat': 37.8806341, 'lng': -84.5729961}
Working response for Nicholasville, KY, USA with results: 37.8806341, -84.5729961
{'lat': 35.7234734, 'lng': -79.4622431}
Working response for Siler City, NC, USA with results: 35.7234734, -79.4622431
{'lat': 41.91252859999999, 'lng': -88.13479269999999}
Working response for Carol Stream, IL, USA with results: 41.91252859999999, -88.13479269999999
{'lat': 40.2005472, 'lng': -74.03141649999999}
Working response for Neptune, NJ, USA with results: 40.2005472, -74.03141649999999
{'lat': 40.3409251, 'lng': -76.4113497}
Working response for Lebanon, PA, USA with results: 40.3409251, -76.4113497
{'lat': 42.9133602, 'lng': -85.7053085}
Working response for Wyoming, MI, USA with results: 42.9133602, -85.7053085
{'lat': 41.506

{'lat': 48.5342662, 'lng': -123.0171242}
Working response for Friday Harbor, WA, USA with results: 48.5342662, -123.0171242
{'lat': 46.2305049, 'lng': -119.0922316}
Working response for Pasco, WA, USA with results: 46.2305049, -119.0922316
{'lat': 30.4548451, 'lng': -97.6222665}
Working response for Pflugerville, TX, USA with results: 30.4548451, -97.6222665
{'lat': 41.6883646, 'lng': -93.7925199}
Working response for Grimes, IA, USA with results: 41.6883646, -93.7925199
{'lat': 36.9759345, 'lng': -82.5757108}
Working response for Wise, VA, USA with results: 36.9759345, -82.5757108
{'lat': 25.7059354, 'lng': -80.35227499999999}
Working response for Sunset, FL, USA with results: 25.7059354, -80.35227499999999
{'lat': 44.7077106, 'lng': -100.0617352}
Working response for Onida, SD, USA with results: 44.7077106, -100.0617352
{'lat': 27.9462615, 'lng': -82.2420317}
Working response for Valrico, FL, USA with results: 27.9462615, -82.2420317
{'lat': 41.878711, 'lng': -71.38255579999999}
Work

Working response for Cedar City, UT, USA with results: 37.67747689999999, -113.0618931
{'lat': 38.403903, 'lng': -96.18166260000001}
Working response for Emporia, KS, USA with results: 38.403903, -96.18166260000001
{'lat': 38.4192496, 'lng': -82.44515400000002}
Working response for Huntington, WV, USA with results: 38.4192496, -82.44515400000002
{'lat': 42.587915, 'lng': -72.5994104}
Working response for Greenfield, MA, USA with results: 42.587915, -72.5994104
{'lat': 44.01690139999999, 'lng': -107.9553721}
Working response for Worland, WY, USA with results: 44.01690139999999, -107.9553721
{'lat': 35.8968236, 'lng': -77.5358049}
Working response for Tarboro, NC, USA with results: 35.8968236, -77.5358049
{'lat': 40.9414874, 'lng': -73.9593041}
Working response for Cresskill, NJ, USA with results: 40.9414874, -73.9593041
{'lat': 41.8939617, 'lng': -87.81984779999999}
Working response for River Forest, IL, USA with results: 41.8939617, -87.81984779999999
{'lat': 34.1805403, 'lng': -118.90

{'lat': 41.1073183, 'lng': -73.7959666}
Working response for Hawthorne, NY, USA with results: 41.1073183, -73.7959666
{'lat': 39.091449, 'lng': -84.4957757}
Working response for Newport, KY, USA with results: 39.091449, -84.4957757
{'lat': 35.0524699, 'lng': -118.1739645}
Working response for Mojave, CA, USA with results: 35.0524699, -118.1739645
{'lat': 34.78384090000001, 'lng': -86.5722237}
Working response for Normal, AL, USA with results: 34.78384090000001, -86.5722237
{'lat': 45.7762463, 'lng': -111.1770945}
Working response for Belgrade, MT, USA with results: 45.7762463, -111.1770945
{'lat': 46.15026599999999, 'lng': -67.6033952}
Working response for Woodstock, NB, CAN with results: 46.15026599999999, -67.6033952
{'lat': 41.84556509999999, 'lng': -71.886358}
Working response for Dayville, CT, USA with results: 41.84556509999999, -71.886358
{'lat': 29.9891052, 'lng': -97.87722670000001}
Working response for Kyle, TX, USA with results: 29.9891052, -97.87722670000001
{'lat': 40.8409

In [47]:
#check that dataframe looks correct
#add info if necessary, or drop items
coords_df
missing_vals = coords_df["Longitude"].isna()
print(coords_df[missing_vals])

     country_code state_code    city Latitude Longitude
1302          CAN         ON   Delta     None      None
1751          CAN         ON  Bolton     None      None


In [49]:
#only two missing values, so I manually got the values for latitude/longitude

#for Bolton, Ontario, Canada:
coords_df["Latitude"][1751] = 43.873021
coords_df["Longitude"][1751] = -79.728130

#for Delta, Ontario, Canada:
coords_df["Latitude"][1302] = 44.608509
coords_df["Longitude"][1302] = -76.123042

#check that all missing values have been filled
missing_vals = coords_df["Longitude"].isna()
print(coords_df[missing_vals])

Empty DataFrame
Columns: [country_code, state_code, city, Latitude, Longitude]
Index: []


In [52]:
#output data from new data frame containing latitudes and longitudes
#to csv in this cell
coords_df.to_csv("./geographical_data/coord_locations.csv", index=False)

In [55]:
#output data to json in this cell - to use for testing Leaflet maps
coords_df.to_json("./geographical_data/coord_locations.json", orient="index")

In [57]:
#look at distribution of cities around the world using gmaps!
#wrong right now because we do not have any values outside of Canada and US

# Configure gmaps
gmaps.configure(api_key=config.API_KEY)

#set locations (lat, long) data for the map
locations = coords_df.loc[:, ["Latitude", "Longitude"]]

# Plot Heatmap
fig = gmaps.figure()

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

## Add more coordinates

* Unfortunately, the earler data was not cleaned correctly since foreign cities were dropped due to not having "state_code" information

* This part of the notebook will add the missing data using the new, more complete cleaned data available at VC_Data/Cleaned_Data

* Data that already exists should not be deleted or re-added (just append new data)

In [4]:
#test that everything is working correctly on an example request
#should look the same as above but not have a state code
base_url = "https://maps.googleapis.com/maps/api/geocode/json?address="

city = "Las Condes"
country_code = "CHL"

response = requests.get("{}{}+{}&key={}".format(base_url, city, country_code, config.API_KEY))

print(json.dumps(response.json(), indent=4))
print(response.json()["results"][0]["geometry"]["location"])

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "Las Condes",
                    "short_name": "Las Condes",
                    "types": [
                        "administrative_area_level_3",
                        "political"
                    ]
                },
                {
                    "long_name": "Santiago Province",
                    "short_name": "Santiago Province",
                    "types": [
                        "administrative_area_level_2",
                        "political"
                    ]
                },
                {
                    "long_name": "Santiago Metropolitan Region",
                    "short_name": "Santiago Metropolitan Region",
                    "types": [
                        "administrative_area_level_1",
                        "political"
                    ]
                },
                {
                    "long_name": "Chile

In [5]:
#read in CSV containing unique city, state, and country data
#these values already have lat/lng info now, so remove from cleaned data
#and then get coordinates for remaining city/country combinations
valid_locations_df = pd.read_csv("./geographical_data/all_places.csv")
valid_locations_df

,country_code,state_code,city
0,USA,NY,New York
1,USA,IL,Champaign
2,USA,CA,Los Angeles
3,USA,NJ,Iselin
4,USA,MD,Woodbine
...,...,...,...
2168,CAN,ON,Paris
2169,USA,FL,Atlantic Beach
2170,USA,OR,Salem
2171,USA,WA,Auburn


In [6]:
cleaned_data_locations_df = pd.read_csv("../VC_Data/Cleaned_Data/cleaned_data.csv")
remaining_locations = cleaned_data_locations_df.loc[:, ["country_code", "city"]]
remaining_locations

,country_code,city
0,USA,New York
1,USA,Los Angeles
2,EST,Tallinn
3,GBR,London
4,USA,Fort Worth
...,...,...
36094,AUT,Gerasdorf Bei Wien
36095,GBR,London
36096,CHN,Beijing
36097,HRV,Split


In [7]:
#code to filter out the duplicate columns based on Stack Overflow answer
#for set difference in pandas
remaining_locations =remaining_locations.merge(valid_locations_df, indicator=True, how="left")[lambda x: x._merge=='left_only'].drop('_merge',1)
remaining_locations

,country_code,city,state_code
2,EST,Tallinn,NaN
3,GBR,London,NaN
5,USA,Oakland Park,NaN
6,ARG,Buenos Aires,NaN
7,HKG,Hong Kong,NaN
...,...,...,...
43981,MYS,Damansara New Village,NaN
43985,AUT,Gerasdorf Bei Wien,NaN
43986,GBR,London,NaN
43987,CHN,Beijing,NaN


In [8]:
remaining_locations = remaining_locations.drop(columns=["state_code"])
remaining_locations

,country_code,city
2,EST,Tallinn
3,GBR,London
5,USA,Oakland Park
6,ARG,Buenos Aires
7,HKG,Hong Kong
...,...,...
43981,MYS,Damansara New Village
43985,AUT,Gerasdorf Bei Wien
43986,GBR,London
43987,CHN,Beijing


In [10]:
#remove duplicates from the remaining locations
remaining_locations = remaining_locations.drop_duplicates()
remaining_locations

,country_code,city
2,EST,Tallinn
3,GBR,London
5,USA,Oakland Park
6,ARG,Buenos Aires
7,HKG,Hong Kong
...,...,...
43862,IND,Jodhpur
43896,ISR,Kefar Sava
43920,ITA,Prato
43952,IND,Ahmadabad


In [11]:
#add latitude, longitude columns
remaining_locations["Latitude"] = None
remaining_locations["Longitude"] = None
remaining_locations

,country_code,city,Latitude,Longitude
2,EST,Tallinn,None,None
3,GBR,London,None,None
5,USA,Oakland Park,None,None
6,ARG,Buenos Aires,None,None
7,HKG,Hong Kong,None,None
...,...,...,...,...
43862,IND,Jodhpur,None,None
43896,ISR,Kefar Sava,None,None
43920,ITA,Prato,None,None
43952,IND,Ahmadabad,None,None


In [13]:
#reset the index
remaining_locations = remaining_locations.reset_index(drop=True)
remaining_locations

,country_code,city,Latitude,Longitude
0,EST,Tallinn,None,None
1,GBR,London,None,None
2,USA,Oakland Park,None,None
3,ARG,Buenos Aires,None,None
4,HKG,Hong Kong,None,None
...,...,...,...,...
1954,IND,Jodhpur,None,None
1955,ISR,Kefar Sava,None,None
1956,ITA,Prato,None,None
1957,IND,Ahmadabad,None,None


In [18]:
#write a function to send all requests - based on previous for loop!

#write for loop to send requests each second, and append data to a copy of the read dataframe
def getCoords(coords_df):
    total_locations = len(coords_df)
    for i in range(total_locations):
        city = coords_df.iloc[i]["city"]
#         state = coords_df.iloc[i]["state_code"]
        country = coords_df.iloc[i]["country_code"]
        try:
            response = requests.get("{}{}+{}&key={}".format(base_url, city, country, config.API_KEY))
            results = response.json()
            print(results["results"][0]["geometry"]["location"])
            lat = results["results"][0]["geometry"]["location"]["lat"]
            lng = results["results"][0]["geometry"]["location"]["lng"]
            #add back if necessary!!! just slows down requests but may not be necessary
#             time.sleep(1)
            coords_df["Latitude"][i] = lat
            coords_df["Longitude"][i] = lng
            print("Working response for {}, {} with results: {}, {}".format(city, country, lat, lng))
            print("{} of {} records".format(i+1, total_locations))
        except:
            print("Issue with {}, {}".format(city, country))


In [19]:
#get coordinate info
getCoords(remaining_locations)

{'lat': 59.43696079999999, 'lng': 24.7535747}
Working response for Tallinn, EST with results: 59.43696079999999, 24.7535747
1 of 1959 records
{'lat': 51.5073509, 'lng': -0.1277583}
Working response for London, GBR with results: 51.5073509, -0.1277583
2 of 1959 records
{'lat': 26.1723065, 'lng': -80.1319893}
Working response for Oakland Park, USA with results: 26.1723065, -80.1319893
3 of 1959 records
{'lat': -34.6036844, 'lng': -58.3815591}
Working response for Buenos Aires, ARG with results: -34.6036844, -58.3815591
4 of 1959 records
{'lat': 22.308047, 'lng': 113.9184808}
Working response for Hong Kong, HKG with results: 22.308047, 113.9184808
5 of 1959 records
{'lat': -33.4079424, 'lng': -70.5451063}
Working response for Las Condes, CHL with results: -33.4079424, -70.5451063
6 of 1959 records
{'lat': 48.856614, 'lng': 2.3522219}
Working response for Paris, FRA with results: 48.856614, 2.3522219
7 of 1959 records
{'lat': 39.90419989999999, 'lng': 116.4073963}
Working response for Beij

{'lat': 31.9730015, 'lng': 34.7925013}
Working response for Rishon Letsiyon, ISR with results: 31.9730015, 34.7925013
66 of 1959 records
{'lat': 19.0759837, 'lng': 72.8776559}
Working response for Mumbai, IND with results: 19.0759837, 72.8776559
67 of 1959 records
{'lat': 42.9764027, 'lng': -88.1084224}
Working response for New Berlin, USA with results: 42.9764027, -88.1084224
68 of 1959 records
{'lat': 51.454513, 'lng': -2.58791}
Working response for Bristol, GBR with results: 51.454513, -2.58791
69 of 1959 records
{'lat': 41.805699, 'lng': 123.431472}
Working response for Shenyang, CHN with results: 41.805699, 123.431472
70 of 1959 records
{'lat': 11.1085242, 'lng': 77.3410656}
Working response for Tirupur, IND with results: 11.1085242, 77.3410656
71 of 1959 records
{'lat': 59.9310584, 'lng': 30.3609096}
Working response for Saint Petersburg, RUS with results: 59.9310584, 30.3609096
72 of 1959 records
{'lat': -23.5015299, 'lng': -47.45256029999999}
Working response for Sorocaba, BRA 

{'lat': 59.22053689999999, 'lng': 10.9347012}
Working response for Fredrikstad, NOR with results: 59.22053689999999, 10.9347012
131 of 1959 records
{'lat': 50.62925, 'lng': 3.057256}
Working response for Lille, FRA with results: 50.62925, 3.057256
132 of 1959 records
{'lat': -33.4488897, 'lng': -70.6692655}
Working response for Santiago, CHL with results: -33.4488897, -70.6692655
133 of 1959 records
{'lat': 43.8486707, 'lng': -73.4234531}
Working response for Ticonderoga, USA with results: 43.8486707, -73.4234531
134 of 1959 records
{'lat': 43.439236, 'lng': -1.590062}
Working response for Bidart, FRA with results: 43.439236, -1.590062
135 of 1959 records
{'lat': 18.7939054, 'lng': 73.334643}
Working response for Khopoli, IND with results: 18.7939054, 73.334643
136 of 1959 records
{'lat': 39.4699075, 'lng': -0.3762881}
Working response for Valencia, ESP with results: 39.4699075, -0.3762881
137 of 1959 records
{'lat': 41.0573188, 'lng': -74.1409771}
Working response for Ramsey, USA with

{'lat': -6.2087634, 'lng': 106.845599}
Working response for Jakarta, IDN with results: -6.2087634, 106.845599
195 of 1959 records
{'lat': 52.01157689999999, 'lng': 4.3570677}
Working response for Delft, NLD with results: 52.01157689999999, 4.3570677
196 of 1959 records
{'lat': 41.9794005, 'lng': 2.8214264}
Working response for Girona, ESP with results: 41.9794005, 2.8214264
197 of 1959 records
{'lat': 36.0770048, 'lng': -87.3877899}
Working response for Dickson, USA with results: 36.0770048, -87.3877899
198 of 1959 records
{'lat': 30.175391, 'lng': -85.632894}
Working response for Panama City, PAN with results: 30.175391, -85.632894
199 of 1959 records
{'lat': 57.70887, 'lng': 11.97456}
Working response for Gothenburg, SWE with results: 57.70887, 11.97456
200 of 1959 records
{'lat': -27.5758579, 'lng': 153.0921619}
Working response for Eight Mile Plains, AUS with results: -27.5758579, 153.0921619
201 of 1959 records
{'lat': 48.730756, 'lng': 2.27137}
Working response for Massy, FRA wit

{'lat': 29.3013479, 'lng': -94.7976958}
Working response for Galveston, USA with results: 29.3013479, -94.7976958
257 of 1959 records
{'lat': 46.54885729999999, 'lng': 6.674199}
Working response for Epalinges, CHE with results: 46.54885729999999, 6.674199
258 of 1959 records
{'lat': 33.9842691, 'lng': 35.6344491}
Working response for Joünié, LBN with results: 33.9842691, 35.6344491
259 of 1959 records
{'lat': 44.4689738, 'lng': 11.4218161}
Working response for San Lazzaro, ITA with results: 44.4689738, 11.4218161
260 of 1959 records
{'lat': 43.2630126, 'lng': -2.9349852}
Working response for Bilbao, ESP with results: 43.2630126, -2.9349852
261 of 1959 records
{'lat': 20.2960587, 'lng': 85.8245398}
Working response for Bhubaneswar, IND with results: 20.2960587, 85.8245398
262 of 1959 records
{'lat': -37.852, 'lng': 144.998}
Working response for Prahran, AUS with results: -37.852, 144.998
263 of 1959 records
{'lat': 46.5296363, 'lng': 6.561525199999999}
Working response for Ecublens, CHE

...

{'lat': 48.906535, 'lng': 2.244085}
Working response for La Garenne-colombes, FRA with results: 48.906535, 2.244085
1702 of 1959 records
{'lat': 25.0295177, 'lng': 55.1836195}
Working response for Dubai, IND with results: 25.0295177, 55.1836195
1703 of 1959 records
{'lat': 50.999041, 'lng': -0.106333}
Working response for Haywards Heath, GBR with results: 50.999041, -0.106333
1704 of 1959 records
{'lat': 53.400091, 'lng': -0.826004}
Working response for Beckingham, GBR with results: 53.400091, -0.826004
1705 of 1959 records
{'lat': 32.0852999, 'lng': 34.78176759999999}
Working response for Tel Mond, ISR with results: 32.0852999, 34.78176759999999
1706 of 1959 records
{'lat': 34.6900806, 'lng': 135.1956311}
Working response for Kobe, JPN with results: 34.6900806, 135.1956311
1707 of 1959 records
{'lat': 38.40866159999999, 'lng': -89.9920501}
Working response for Smithton, USA with results: 38.40866159999999, -89.9920501
1708 of 1959 records
{'lat': 41.6060873, 'lng': -93.77235999999999}

{'lat': 27.0410218, 'lng': 88.2662745}
Working response for Darjeeling, IND with results: 27.0410218, 88.2662745
1764 of 1959 records
{'lat': 51.54128040000001, 'lng': 9.915803499999999}
Working response for Göttingen, DEU with results: 51.54128040000001, 9.915803499999999
1765 of 1959 records
{'lat': 42.7404881, 'lng': 12.7378003}
Working response for Spoleto, ITA with results: 42.7404881, 12.7378003
1766 of 1959 records
{'lat': -26.9165792, 'lng': -49.0717331}
Working response for Blumenau, BRA with results: -26.9165792, -49.0717331
1767 of 1959 records
{'lat': 36.5270612, 'lng': -6.2885962}
Working response for Cadiz, ESP with results: 36.5270612, -6.2885962
1768 of 1959 records
{'lat': 47.4695081, 'lng': -0.6234109999999999}
Working response for Beaucouzé, FRA with results: 47.4695081, -0.6234109999999999
1769 of 1959 records
{'lat': 45.73673369999999, 'lng': 13.7490015}
Working response for Sgonico, ITA with results: 45.73673369999999, 13.7490015
1770 of 1959 records
{'lat': 43.40

{'lat': 32.321458, 'lng': 34.853196}
Working response for Natanya, ISR with results: 32.321458, 34.853196
1825 of 1959 records
{'lat': 41.1224394, 'lng': 25.4065582}
Working response for Komotiní, GRC with results: 41.1224394, 25.4065582
1826 of 1959 records
{'lat': 51.0432117, 'lng': 3.7592682}
Working response for Gentbrugge, BEL with results: 51.0432117, 3.7592682
1827 of 1959 records
{'lat': 52.6275324, 'lng': 4.9449451}
Working response for De Goorn, NLD with results: 52.6275324, 4.9449451
1828 of 1959 records
{'lat': 48.7262433, 'lng': 2.3652472}
Working response for Orly, FRA with results: 48.7262433, 2.3652472
1829 of 1959 records
{'lat': 36.140751, 'lng': -5.353585}
Working response for Gibraltar, GIB with results: 36.140751, -5.353585
1830 of 1959 records
{'lat': 55.420557, 'lng': 9.100062000000001}
Working response for Københoved, DNK with results: 55.420557, 9.100062000000001
1831 of 1959 records
{'lat': 42.69376339999999, 'lng': -120.5460827}
Working response for Paisley, 

{'lat': 41.490047, 'lng': 2.1397612}
Working response for Cerdañola, ESP with results: 41.490047, 2.1397612
1888 of 1959 records
{'lat': 56.01528339999999, 'lng': 92.8932476}
Working response for Krasnoyarsk, RUS with results: 56.01528339999999, 92.8932476
1889 of 1959 records
{'lat': -30.0346316, 'lng': -51.2176986}
Working response for Rio Grande Do Sul, BRA with results: -30.0346316, -51.2176986
1890 of 1959 records
{'lat': 32.3476438, 'lng': -97.38668369999999}
Working response for Cleburne, USA with results: 32.3476438, -97.38668369999999
1891 of 1959 records
{'lat': 45.9068105, 'lng': 12.1297097}
Working response for Farra Di Soligo, ITA with results: 45.9068105, 12.1297097
1892 of 1959 records
{'lat': 32.0852999, 'lng': 34.78176759999999}
Working response for Tell Abib, ISR with results: 32.0852999, 34.78176759999999
1893 of 1959 records
{'lat': 50.595484, 'lng': -1.2061}
Working response for Ventnor, GBR with results: 50.595484, -1.2061
1894 of 1959 records
{'lat': 48.118146, '

{'lat': 50.2400205, 'lng': 4.7612111}
Working response for Anthée, BEL with results: 50.2400205, 4.7612111
1950 of 1959 records
{'lat': 45.899247, 'lng': 6.129384}
Working response for Annecy, FRA with results: 45.899247, 6.129384
1951 of 1959 records
{'lat': 42.098408, 'lng': -76.04936839999999}
Working response for Endicott, USA with results: 42.098408, -76.04936839999999
1952 of 1959 records
{'lat': 31.654375, 'lng': 120.752481}
Working response for Changshu, CHN with results: 31.654375, 120.752481
1953 of 1959 records
{'lat': 9.0798183, 'lng': 7.4922407}
Working response for Wuse, NGA with results: 9.0798183, 7.4922407
1954 of 1959 records
{'lat': 26.2389469, 'lng': 73.02430939999999}
Working response for Jodhpur, IND with results: 26.2389469, 73.02430939999999
1955 of 1959 records
{'lat': 32.178195, 'lng': 34.90761}
Working response for Kefar Sava, ISR with results: 32.178195, 34.90761
1956 of 1959 records
{'lat': 43.8777049, 'lng': 11.102228}
Working response for Prato, ITA with 

In [21]:
#check how many locations had issues
missing_vals = remaining_locations["Longitude"].isna()
print(remaining_locations[missing_vals])

     country_code                   city Latitude Longitude
18            BEL              Ferrières     None      None
96            BRA                    Itá     None      None
122           BRA                Olímpia     None      None
164           HRV                   Nova     None      None
180           DEU                Deutsch     None      None
266           GRC                Maroúli     None      None
321           CHE                   Baar     None      None
374           ESP                  Leiva     None      None
408           RUS                 Frunze     None      None
412           ESP              Santander     None      None
445           TUR                 Finans     None      None
465           ITA                   Cona     None      None
521           IRL              Blackrock     None      None
588           SLV              Mexicanos     None      None
657           AUT                  Brand     None      None
706           FRA                  Orsay

In [40]:
#create a new df for the missing values, then add the values
patch_coords = remaining_locations[missing_vals]
patch_coords = patch_coords.reset_index(drop=True)
patch_coords

,country_code,city,Latitude,Longitude
0,BEL,Ferrières,None,None
1,BRA,Itá,None,None
2,BRA,Olímpia,None,None
3,HRV,Nova,None,None
4,DEU,Deutsch,None,None
5,GRC,Maroúli,None,None
6,CHE,Baar,None,None
7,ESP,Leiva,None,None
8,RUS,Frunze,None,None
9,ESP,Santander,None,None


In [41]:
patch_coords = patch_coords.merge(cleaned_data_locations_df, how = "left")
patch_coords = patch_coords.loc[:, ["country_code", "region", "city", "Latitude", "Longitude"]]
patch_coords = patch_coords.drop_duplicates()
patch_coords = patch_coords.reset_index(drop=True)
patch_coords

,country_code,region,city,Latitude,Longitude
0,BEL,BEL - Other,Ferrières,None,None
1,BRA,Itá,Itá,None,None
2,BRA,BRA - Other,Olímpia,None,None
3,HRV,HRV - Other,Nova,None,None
4,DEU,DEU - Other,Deutsch,None,None
5,GRC,Maroúli,Maroúli,None,None
6,CHE,Baar,Baar,None,None
7,ESP,ESP - Other,Leiva,None,None
8,RUS,RUS - Other,Frunze,None,None
9,ESP,Santander,Santander,None,None


In [42]:
#see what went wrong
#with a manual search

city = "Clever"
region = "MO - Other"
country_code = "USA"

response = requests.get("{}{}+{}+{}&key={}".format(base_url, city, region, country_code, config.API_KEY))

print(json.dumps(response.json(), indent=4))
print(response.json()["results"][0]["geometry"]["location"])

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "Clever",
                    "short_name": "Clever",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "Lincoln Township",
                    "short_name": "Lincoln Township",
                    "types": [
                        "administrative_area_level_3",
                        "political"
                    ]
                },
                {
                    "long_name": "Christian County",
                    "short_name": "Christian County",
                    "types": [
                        "administrative_area_level_2",
                        "political"
                    ]
                },
                {
                    "long_name": "Missouri",
                    "short_name": "MO",
       

In [60]:
#get coordinates for patched values
def getCoords_Region(coords_df):
    total_locations = len(coords_df)
    for i in range(total_locations):
        city = coords_df.iloc[i]["city"]
#         region = coords_df.iloc[i]["region"]
        country = coords_df.iloc[i]["country_code"].lower()
        print(country)
        country = pycountry.countries.get(alpha_3=country).name
        print(country)
        try:
            response = requests.get("{}{}+{}&key={}".format(base_url, city, country, config.API_KEY))
            results = response.json()
            print(results["results"][0]["geometry"]["location"])
            lat = results["results"][0]["geometry"]["location"]["lat"]
            lng = results["results"][0]["geometry"]["location"]["lng"]
            #add back if necessary!!! just slows down requests but may not be necessary
#             time.sleep(1)
            coords_df["Latitude"][i] = lat
            coords_df["Longitude"][i] = lng
            print("Working response for {}, {} with results: {}, {}".format(city, country, lat, lng))
            print("{} of {} records".format(i+1, total_locations))
        except:
            print("Issue with {}, {}".format(city, country))

In [61]:
getCoords_Region(patch_coords)

bel
Belgium
{'lat': 50.40029, 'lng': 5.60564}
Working response for Ferrières, Belgium with results: 50.40029, 5.60564
1 of 42 records
bra
Brazil
{'lat': -23.2108714, 'lng': -45.8753055}
Working response for Itá, Brazil with results: -23.2108714, -45.8753055
2 of 42 records
bra
Brazil
{'lat': -20.7371116, 'lng': -48.9110606}
Working response for Olímpia, Brazil with results: -20.7371116, -48.9110606
3 of 42 records
hrv
Croatia
{'lat': 45.3452368, 'lng': 14.3098232}
Working response for Nova, Croatia with results: 45.3452368, 14.3098232
4 of 42 records
deu
Germany
{'lat': 51.165691, 'lng': 10.451526}
Working response for Deutsch, Germany with results: 51.165691, 10.451526
5 of 42 records
grc
Greece
{'lat': 38.8692414, 'lng': 23.241735}
Working response for Maroúli, Greece with results: 38.8692414, 23.241735
6 of 42 records
che
Switzerland
{'lat': 47.1953729, 'lng': 8.526087}
Working response for Baar, Switzerland with results: 47.1953729, 8.526087
7 of 42 records
esp
Spain
{'lat': 42.502

In [62]:
patch_coords

,country_code,region,city,Latitude,Longitude
0,BEL,BEL - Other,Ferrières,50.4003,5.60564
1,BRA,Itá,Itá,-23.2109,-45.8753
2,BRA,BRA - Other,Olímpia,-20.7371,-48.9111
3,HRV,HRV - Other,Nova,45.3452,14.3098
4,DEU,DEU - Other,Deutsch,51.1657,10.4515
5,GRC,Maroúli,Maroúli,38.8692,23.2417
6,CHE,Baar,Baar,47.1954,8.52609
7,ESP,ESP - Other,Leiva,42.5026,-3.04698
8,RUS,RUS - Other,Frunze,None,None
9,ESP,Santander,Santander,43.4623,-3.80998


In [64]:
#check how many locations had issues
missing_vals2 = patch_coords["Longitude"].isna()
print(patch_coords[missing_vals2])

   country_code       region    city Latitude Longitude
8           RUS  RUS - Other  Frunze     None      None
19          CHN  CHN - Other   Weiji     None      None
38          DEU  DEU - Other  Albert     None      None


In [65]:
#have only 3 missing values now, find those manually
last_patch = patch_coords[missing_vals2].reset_index(drop=True)
last_patch

,country_code,region,city,Latitude,Longitude
0,RUS,RUS - Other,Frunze,None,None
1,CHN,CHN - Other,Weiji,None,None
2,DEU,DEU - Other,Albert,None,None


In [68]:
#get Frunze coordinates
last_patch["Latitude"][0] = 63.070365
last_patch["Longitude"][0] = 129.728320

#get Weiji coordinates
last_patch["Latitude"][1] = 30.691526
last_patch["Longitude"][1] = 98.392772

In [67]:
#issues finding "Albert" in Germany, so lookup here:
cleaned_data_locations_df.loc[cleaned_data_locations_df["city"] == "Albert"]

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,region,city,...,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
31864,/organization/toprealty,TopRealty,http://www.toprealty.ca,|Real Estate|,Real Estate,315000,operating,DEU,DEU - Other,Albert,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
#actually in Grand Prarie, Alberta, Canada according to website
#revise here, but note that some data ay be inaccurate
#need to look at urls to make sure country codes match the country possibly in the future

#get Albert coordinates
last_patch["Latitude"][2] = 55.170684
last_patch["Longitude"][2] = -118.800859
last_patch["country_code"][2] = "CAN"
last_patch["city"][2] = "Grand Prairie"
last_patch["region"][2] = "Alberta"

last_patch

,country_code,region,city,Latitude,Longitude
0,RUS,RUS - Other,Frunze,63.0704,129.728
1,CHN,CHN - Other,Weiji,30.6915,98.3928
2,CAN,Alberta,Grand Prairie,55.1707,-118.801


In [70]:
#now add all iteratively added values to the output sheet
#first, get geographical data sheet
previous_data = pd.read_csv("geographical_data/coord_locations.csv")
previous_data

,country_code,state_code,city,Latitude,Longitude
0,USA,NY,New York,40.712775,-74.005973
1,USA,IL,Champaign,40.116420,-88.243383
2,USA,CA,Los Angeles,34.052234,-118.243685
3,USA,NJ,Iselin,40.575382,-74.322370
4,USA,MD,Woodbine,39.373817,-77.056904
...,...,...,...,...,...
2168,CAN,ON,Paris,43.194020,-80.384500
2169,USA,FL,Atlantic Beach,30.334408,-81.398696
2170,USA,OR,Salem,42.519540,-70.896715
2171,USA,WA,Auburn,47.307323,-122.228453


In [72]:
#no longer need the state code, so drop it!
previous_data = previous_data.drop(columns=["state_code"])
previous_data

,country_code,city,Latitude,Longitude
0,USA,New York,40.712775,-74.005973
1,USA,Champaign,40.116420,-88.243383
2,USA,Los Angeles,34.052234,-118.243685
3,USA,Iselin,40.575382,-74.322370
4,USA,Woodbine,39.373817,-77.056904
...,...,...,...,...
2168,CAN,Paris,43.194020,-80.384500
2169,USA,Atlantic Beach,30.334408,-81.398696
2170,USA,Salem,42.519540,-70.896715
2171,USA,Auburn,47.307323,-122.228453


In [73]:
#now drop nulls in remaining_locations
remaining_locations = remaining_locations.dropna()
remaining_locations

,country_code,city,Latitude,Longitude
0,EST,Tallinn,59.437,24.7536
1,GBR,London,51.5074,-0.127758
2,USA,Oakland Park,26.1723,-80.132
3,ARG,Buenos Aires,-34.6037,-58.3816
4,HKG,Hong Kong,22.308,113.918
...,...,...,...,...
1953,NGA,Wuse,9.07982,7.49224
1954,IND,Jodhpur,26.2389,73.0243
1955,ISR,Kefar Sava,32.1782,34.9076
1956,ITA,Prato,43.8777,11.1022


In [76]:
#now drop extra columns and nulls in patch_coords dataframe
selected_cols = ["country_code", "city", "Latitude", "Longitude"]
patch_coords = patch_coords.loc[:, selected_cols]
patch_coords = patch_coords.dropna()
patch_coords

,country_code,city,Latitude,Longitude
0,BEL,Ferrières,50.4003,5.60564
1,BRA,Itá,-23.2109,-45.8753
2,BRA,Olímpia,-20.7371,-48.9111
3,HRV,Nova,45.3452,14.3098
4,DEU,Deutsch,51.1657,10.4515
5,GRC,Maroúli,38.8692,23.2417
6,CHE,Baar,47.1954,8.52609
7,ESP,Leiva,42.5026,-3.04698
9,ESP,Santander,43.4623,-3.80998
10,TUR,Finans,38.6833,35.5667


In [77]:
#drop extra columns for final patch values
last_patch = last_patch.loc[:, selected_cols]
last_patch

,country_code,city,Latitude,Longitude
0,RUS,Frunze,63.0704,129.728
1,CHN,Weiji,30.6915,98.3928
2,CAN,Grand Prairie,55.1707,-118.801


In [78]:
#append all new values to original .csv dataframe
output_data = previous_data.append(remaining_locations, ignore_index=True)
output_data = output_data.append(patch_coords, ignore_index=True)
output_data = output_data.append(last_patch, ignore_index=True)
output_data

,country_code,city,Latitude,Longitude
0,USA,New York,40.7128,-74.006
1,USA,Champaign,40.1164,-88.2434
2,USA,Los Angeles,34.0522,-118.244
3,USA,Iselin,40.5754,-74.3224
4,USA,Woodbine,39.3738,-77.0569
...,...,...,...,...
4127,SYC,Victoria,-4.61914,55.4513
4128,MYS,Damansara New Village,3.14215,101.65
4129,RUS,Frunze,63.0704,129.728
4130,CHN,Weiji,30.6915,98.3928


In [79]:
output_data = output_data.drop_duplicates()
output_data

,country_code,city,Latitude,Longitude
0,USA,New York,40.7128,-74.006
1,USA,Champaign,40.1164,-88.2434
2,USA,Los Angeles,34.0522,-118.244
3,USA,Iselin,40.5754,-74.3224
4,USA,Woodbine,39.3738,-77.0569
...,...,...,...,...
4127,SYC,Victoria,-4.61914,55.4513
4128,MYS,Damansara New Village,3.14215,101.65
4129,RUS,Frunze,63.0704,129.728
4130,CHN,Weiji,30.6915,98.3928


In [84]:
output_data = output_data.reset_index(drop=True)
output_data

,country_code,city,Latitude,Longitude
0,USA,New York,40.7128,-74.006
1,USA,Champaign,40.1164,-88.2434
2,USA,Los Angeles,34.0522,-118.244
3,USA,Iselin,40.5754,-74.3224
4,USA,Woodbine,39.3738,-77.0569
...,...,...,...,...
4118,SYC,Victoria,-4.61914,55.4513
4119,MYS,Damansara New Village,3.14215,101.65
4120,RUS,Frunze,63.0704,129.728
4121,CHN,Weiji,30.6915,98.3928


In [86]:
#output the .csv file now with the full data
output_data.to_csv("geographical_data/coord_locations.csv", index=False)

In [87]:
#output the .json file now with the full data
#used for testing/setting up Leaflet maps
output_data.to_json("./geographical_data/coord_locations.json", orient="index")